In visual code create environment
Ctrl+Shift+P
pip install gradio

In [ ]:
import sys
import gradio as gr
import io
import os
import requests
from dotenv import load_dotenv
from PIL import Image

In [ ]:
load_dotenv("azure.env")

azure_cv_key = os.getenv("azure_cv_key")
azure_cv_endpoint = os.getenv("azure_cv_endpoint")



In [ ]:
def post_request(url: str, image_data: bytes, key: str) -> bytes:
    """Sends a POST request to Azure and returns the response content."""
    headers = {
        "Content-type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": key,
    }
    
    try:
        with requests.Session() as session:
            response = session.post(url, data=image_data, headers=headers)
        response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        raise SystemExit(f"An error occurred: {e}")

In [ ]:
def remove_background_from_image(image: Image.Image, endpoint=azure_cv_endpoint, key=azure_cv_key) -> tuple:
    """Remove background from image using Azure Computer Vision."""
    
    base_url = f"{endpoint}/computervision/imageanalysis:segment?api-version=2023-02-01-preview&mode="
    remove_background_url = f"{base_url}backgroundRemoval"
    get_mask_object_url = f"{base_url}foregroundMatting"
    
    with io.BytesIO() as output:
        image.save(output, format="JPEG")
        image_data = output.getvalue()
        
    wo_background_content = post_request(remove_background_url, image_data, key)
    mask_content = post_request(get_mask_object_url, image_data, key)
    
    wo_background_image = Image.open(io.BytesIO(wo_background_content))
    mask_image = Image.open(io.BytesIO(mask_content))

    return wo_background_image, mask_image

In [ ]:
# Gradio UI
logo = "https://assetspost.blob.core.windows.net/png/Logo-YT-PablitoPiova.fw.png"
header = "Background removal with Azure Computer Vision"
image = f"<center> <img src={logo} width=50px></center>"
samples = ["images/piova-01.jpg", "images/piova-02.jpg", "images/piova-03.jpg", "images/mvp.png", "images/MVP-Box.jpg", "images/Xamarin.jpg", "images/Joystick.jpg"]

inputs = gr.Image(type="pil", label="Input Image")
outputs = [gr.Image(label="Image without background", type="pil"), gr.Image(label="Mask", type="pil")]

webapp = gr.Interface(
    remove_background_from_image,
    inputs,
    outputs,
    title=header,
    description=image,
    examples=samples,
    theme="freddyaboulton/dracula_revamped",
)

webapp.launch(share=True)